In [10]:
import glob
import os
import json

ipynbs = glob.glob('**/*.ipynb', recursive=True)
new_ipynbs = []
for ipynb in ipynbs:
    if ipynb == 'ipynb-to-mdx.ipynb':
        continue
    mdx = ipynb.replace('.ipynb', '.mdx')
    source_mtime = os.path.getmtime(ipynb)
    target_mtime = os.path.getmtime(mdx) if os.path.exists(mdx) else 0
    if source_mtime > target_mtime:
        new_ipynbs.append(ipynb)

excludes = ['machine-learning\\DTW.ipynb',
 'machine-learning\\pytorch.ipynb',
 'machine-learning\\Untitled4.ipynb',
 'machine-learning\\Untitled5.ipynb',
 'machine-learning\\Untitled6.ipynb',
 'machine-learning\\Untitled7.ipynb',]

[ipynb for ipynb in new_ipynbs if ipynb not in excludes]

['computer-vision\\annotation.ipynb',
 'computer-vision\\object-detection.ipynb',
 'computer-vision\\transfer-learning.ipynb']

In [11]:
for ipynb in new_ipynbs:
    convert_ipynb(ipynb)

{'name': 'stderr', 'output_type': 'stream', 'text': ["Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.\n"]}
{'name': 'stderr', 'output_type': 'stream', 'text': ["Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']\n", 'You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.\n']}


In [3]:
def convert_ipynb(ipynb):
    with open(ipynb) as f:
        data = json.load(f)

    lines = []
    for cell in data['cells']:
        lines.append('\n\n')
        if cell['cell_type'] == 'code':
            SKIP_ERROR = False
            if 'source' not in cell or not cell['source'] or cell['source'][0].startswith('#!hide'):
                continue

            # 셀 처리 명령
            SKIP_ERROR = SKIP_STDOUT = SKIP_STREAM = False
            if cell['source'][0].startswith('#!'):
                commands = cell['source'][0][2:].strip().split()
                cell['source'] = cell['source'][1:]
                SKIP_ERROR = 'skip_error' in commands
                SKIP_STDOUT = 'skip_stdout' in commands
                SKIP_STREAM = 'skip_stream' in commands

            # 코드
            lines.append('\n```python copy\n')
            lines.extend(cell['source'])
            lines.append('\n```\n')
            
            # 출력
            if (outputs := cell.get('outputs')):
                output_lines = []
                for output in outputs:
                    if output['output_type'] == 'stream' and not SKIP_STREAM:
                        if output['name'] == 'stderr' and SKIP_ERROR:
                            continue
                        output_lines.append('\n```\n')
                        output_lines.extend(output['text'])
                        output_lines.append('\n```\n')
                    if 'data' in output:
                        if 'text/plain' in output['data']:
                            output_lines.append('\n```\n')
                            output_lines.extend(output['data']['text/plain'])
                            output_lines.append('\n```\n')
                        if 'image/png' in output['data']:
                            output_lines.append('\n![](data:image/png;base64,' + output['data']['image/png'] + ')\n')
                        
                    else:
                        print(output)
                if output_lines:
                    lines.append('\n:::info[output]')
                    lines.extend(output_lines)
                    lines.append('\n:::\n')

        elif cell['cell_type'] == 'markdown':
            lines.extend(cell['source'])   


    with open(ipynb.replace('.ipynb', '.mdx'), 'w') as f:
        f.write(''.join(lines))